In [3]:
import os
os.getcwd()
os.chdir("C:\\Users\\baris\\Desktop\\Modern Data Analytics\\mda_project")

In [11]:
import pandas as pd
import numpy as np

'LOAD HEAT WAVE DATA'
heat_wave = pd.read_csv("out/mortality_heat_wave.csv", sep = ",")
print(heat_wave)

'LOAD DEATH DATA WITH TOTAL WEEKLY POPULATION BY ARRONDISSEMENT (AS OBTAINED BY LINEAR INTERPOLATION)'
mort_interp = pd.read_csv("out/weekly_mortality_with_linear_interpolated_population_data.csv",
                         sep = ",")
print(mort_interp)
mort_interp.info()

       Unnamed: 0 YEAR_WEEK  NIS_Code Nom_arrondissement  \
0               0  2000-001     11000          Antwerpen   
1               1  2000-001     11000          Antwerpen   
2               2  2000-002     11000          Antwerpen   
3               3  2000-002     11000          Antwerpen   
4               4  2000-003     11000          Antwerpen   
5               5  2000-003     11000          Antwerpen   
6               6  2000-004     11000          Antwerpen   
7               7  2000-004     11000          Antwerpen   
8               8  2000-005     11000          Antwerpen   
9               9  2000-005     11000          Antwerpen   
10             10  2000-006     11000          Antwerpen   
11             11  2000-006     11000          Antwerpen   
12             12  2000-007     11000          Antwerpen   
13             13  2000-007     11000          Antwerpen   
14             14  2000-008     11000          Antwerpen   
15             15  2000-008     11000   

FileNotFoundError: [Errno 2] File b'out/weekly_mortality_with_linear_interpolated_population_data.csv' does not exist: b'out/weekly_mortality_with_linear_interpolated_population_data.csv'

In [12]:
'FILTER HEAT_WAVE DATA TO GET ONLY NATURAL DEATHS'
heat_wave = heat_wave[heat_wave['COD'] == "natural"]

'CONVERT YEAR_WEEK TO STRING'
heat_wave = heat_wave.astype({"YEAR_WEEK": str})

'CREATING YEAR AND WEEK VARIABLES'
heat_wave['YEAR'] = heat_wave['YEAR_WEEK'].astype(str).str[:4]
heat_wave['YEAR'] = heat_wave['YEAR'].astype(int)

heat_wave['WEEK'] = heat_wave['YEAR_WEEK'].astype(str).str[6:8]
heat_wave['WEEK'] = heat_wave['WEEK'].astype(int)

print(heat_wave)
heat_wave.info()

       Unnamed: 0 YEAR_WEEK  NIS_Code Nom_arrondissement  \
1               1  2000-001     11000          Antwerpen   
3               3  2000-002     11000          Antwerpen   
5               5  2000-003     11000          Antwerpen   
7               7  2000-004     11000          Antwerpen   
9               9  2000-005     11000          Antwerpen   
11             11  2000-006     11000          Antwerpen   
13             13  2000-007     11000          Antwerpen   
15             15  2000-008     11000          Antwerpen   
17             17  2000-009     11000          Antwerpen   
19             19  2000-010     11000          Antwerpen   
21             21  2000-011     11000          Antwerpen   
23             23  2000-012     11000          Antwerpen   
25             25  2000-013     11000          Antwerpen   
27             27  2000-014     11000          Antwerpen   
29             29  2000-015     11000          Antwerpen   
31             31  2000-016     11000   

In [13]:
heat_wave.head()

,Unnamed: 0,YEAR_WEEK,NIS_Code,Nom_arrondissement,heatwave_week_boolean,ARRON,COD,N_MASK,YEAR,WEEK
1,1,2000-001,11000,Antwerpen,0,11000,natural,31.0,2000,1
3,3,2000-002,11000,Antwerpen,0,11000,natural,263.0,2000,2
5,5,2000-003,11000,Antwerpen,0,11000,natural,219.0,2000,3
7,7,2000-004,11000,Antwerpen,0,11000,natural,226.0,2000,4
9,9,2000-005,11000,Antwerpen,0,11000,natural,229.0,2000,5


In [14]:
'MERGING HEAT_WAVE AND INTERPOLATION DATASETS'

merged_data = pd.merge(mort_interp, heat_wave, how = 'left', left_on=['YEAR', 'ARRON', 'WEEK'], right_on = ['YEAR', 'ARRON', 'WEEK'])
merged_data.head(20)
#merged_data.info()

,YEAR,COD_x,ARRON,N_DEATHS,WEEK,population_t,Unnamed: 0,YEAR_WEEK,NIS_Code,Nom_arrondissement,heatwave_week_boolean,COD_y,N_MASK
0,2000,natural,11000,31.0,1,932103.0,1.0,2000-001,11000.0,Antwerpen,0.0,natural,31.0
1,2000,natural,12000,10.0,1,305670.0,2080.0,2000-001,12000.0,Mechelen,0.0,natural,10.0
2,2000,natural,13000,14.0,1,406199.0,4105.0,2000-001,13000.0,Turnhout,0.0,natural,14.0
3,2000,natural,21000,43.0,1,959318.0,6163.0,2000-001,21000.0,Bruxelles-Capitale,0.0,natural,43.0
4,2000,natural,23000,14.0,1,558220.0,8243.0,2000-001,23000.0,Hal-Vilvorde,0.0,natural,14.0
5,2000,natural,24000,20.0,1,456484.0,10316.0,2000-001,24000.0,Leuven,0.0,natural,20.0
6,2000,natural,25000,15.0,1,349884.0,12379.0,2000-001,25000.0,Nivelles,0.0,natural,15.0
7,2000,natural,31000,14.0,1,270866.0,14441.0,2000-001,31000.0,Brugge,0.0,natural,14.0
8,2000,natural,32000,NaN,1,47992.0,16458.0,2000-001,32000.0,Diksmuide,0.0,natural,NaN
9,2000,natural,33000,6.0,1,104278.0,17936.0,2000-001,33000.0,Ypres,0.0,natural,6.0


In [15]:
'SAVING MERGED DATA TO OUT FILE'
merged_data.to_csv('out/merged.csv', encoding = 'utf-8-sig') 